### Перегрузка типизированной функции
##### Python — очень гибкий язык, однако одна вещь, которую он изначально не допускает, — это перегрузка функций. (За некоторыми исключениями) . Для тех, кто заинтересован, эта существующая ката исследует возможность перегрузки методов объекта для различного количества аргументов. В этом ката мы собираемся реализовать форму перегрузки для любой функции/метода, основанную на классах аргументов, используя декораторы.

#### Обзор
##### Вы будете писать декоратор @overload(*types), который позволит определять функцию или метод несколько раз для разных комбинаций типов входных аргументов. Комбинация и порядок классов аргументов должны вызывать определенную функцию.


#### Основной пример
```
@overload(int)
def analyse(x):
    print('X is an int!')

@overload(list)
def analyse(x):
    print('X is a list!')

analyse([])  # X is a list!
analyse(3)   # X is an int!
```
### Дополнительная информация:
#### Декоратор
* Он должен принимать любой тип или класс, даже созданные пользователем классы. Обратите внимание, что унаследованный класс НЕ считается своим родительским классом. Каждый пользовательский класс следует считать уникальным.
* Он должен иметь возможность обрабатывать любое количество аргументов, включая ноль.
* Если перегруженная функция/метод вызывается без соответствующего набора типов/классов, должно быть возбуждено исключение.
* Декоратор можно использовать как с обычными функциями, так и с методами экземпляра класса (см. ниже о поведении перегрузки и перезаписи). «Волшебные методы» (например __init__, , __call__) НЕ будут тестироваться.

#### Декорированная функция или метод
* Функция/метод должны возвращать правильные значения.
* Все другие обычные функции должны работать (рекурсия, передача функции в качестве аргумента и т. д.).
* Статические методы и методы класса НЕ будут тестироваться.
* Обратите внимание, что при декорировании метода экземпляра self декоратору не предоставляется тип. Декорированная функция должна еще иметь возможность использовать ссылку на self.
* Varargs ( *args) может использоваться в определениях функций. В этом случае доверяйте типам, переданным декоратору, чтобы решить, какой вызов должен быть сопоставлен с этой функцией.
* Вам не нужно учитывать проблемы с областью действия, например наличие двух разных функций с одним и тем же именем, но в двух разных областях. Все функции будут определены как функции верхнего уровня или методы объекта верхнего уровня.

#### Перегрузка и перезапись поведения
* Несколько различных перегруженных функций/методов должны существовать одновременно: func(int,list)перегружает предыдущую func(list)и не перезаписывает ее.
* Функции и методы следует рассматривать как отдельные , т.е. analyse(int)и my_obj.analyse(int)не совпадают, и один не должен перезаписывать другой.
* Если функция/метод с «известным» именем снова украшена уже существующей комбинацией типов, предыдущее определение должно быть перезаписано (см. примеры ниже).

### Более конкретные примеры
##### Должен принимать любой класс или тип. Аргументы переменной длины должны работать:
```
class My_Class: pass
class My_Inherited_Class(My_Class): pass

@overload(int, My_Class, str)
def some_function(*args):
    return 'Option 1'

@overload()
def some_function():
    return 'Option 2'

A = My_Class()
B = My_Inherited_Class()

some_function()               # Option 2
some_function(3, A, 'Words')  # Option 1 <= trust the types given to the decorator
some_function('oops!')        # Exception raised <= no matching types combination
some_function(3, B, 'Words')  # Exception raised <= doesn't consider inheritence
```
##### Должен уметь управлять функциями И методами:
```
@overload(int)
def spin(x):
    return x*3-1

@overload(str)
def spin(x):
    return x[1:] + x[0]

print(spin(6))            # 17
print(spin('Hello'))      # elloH  <=  overload correctly, just like before...
print(spin(''))           # raise IndexError


class T:
    def __init__(self, x):
        self.val = x

    @overload(int)
    def spin(self, x):
        return self.val * x

    @overload(str)
    def spin(self, x):
        return x + str(self.val)

obj = T(7)
print(spin(6))            # 17 <= the instance method doesn't overwrite the function
print(obj.spin(2))        # 14 <= `self` is still usable

print(spin('Hello'))      # elloH
print(obj.spin('Hello'))  # Hello7
```
##### Предыдущие определения должны быть перезаписаны:
```
@overload(str)
def upgrade(x):
    print('First')

@overload(str)
def upgrade(x):
    print('Second')

upgrade('Y')    # Second
```

# Basic Example
@overload(int)
def analyse(x):
    return 'X is an int!'

@overload(list)
def analyse(x):
    return 'X is a list!'

# Should accept class or type
class My_Class:
    pass

class My_Inherited_Class(My_Class):
    pass

@overload(int, My_Class, str)
def some_function(*args):
    return 'Option 1'

@overload()
def some_function():
    return 'Option 2'

# Functions and methods
@overload(int)
def flip(x):
    return 100-x

@overload(str)
def flip(x):
    return x[::-1]

@overload(int)
def spin(x):
    return x*3-1

@overload(str)
def spin(x):
    return x[1:] + x[0]

class T:
    def __init__(self, x):
        self.val = x

    @overload(int)
    def spin(self, x):
        return self.val * x

    @overload(str)
    def spin(self, x):
        return x + str(self.val)

# Overwriting and exceptions
@overload(str)
def upgrade(x):
    return 'First'

@overload()
def upgrade():
    return 'No input'

@overload(str)
def upgrade(x):
    return 'Second'


@test.describe("Example Tests")
def tests():
    @test.it("Basic Example")
    def basic_ex():
        test.assert_equals(analyse([]), 'X is a list!')
        test.assert_equals(analyse(3), 'X is an int!')
    @test.it("Should accept class or type")
    def class_or_type():
        A = My_Class()
        B = My_Inherited_Class()
        arg_list = [1, A, 'Letters']
        test.assert_equals(some_function(3, A, 'Words'), 'Option 1')
        test.assert_equals(some_function(*arg_list), 'Option 1')
        test.assert_equals(some_function(), 'Option 2')
        test.expect_error("Should throw error, function not defined.", lambda: some_function(3, B, 'Words'))
    @test.it("Functions and Methods")
    def fun_and_meth():
        ob = T(7)
        test.assert_equals(flip(6), 94)
        test.assert_equals(spin('Hello'), 'elloH')
        test.assert_equals(ob.spin(2), 14)
    @test.it("Overwriting and Exceptions")
    def over_and_exc():
        test.assert_equals(upgrade('Y'), 'Second')
        test.expect_error("Should throw error, function not defined.", lambda: upgrade(5))

In [ ]:
def overload(*types):
    pass